In [6]:
#Imports
import json
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openweathermapy.core as ow
from citipy import citipy 
import random
import requests as req

#OpenWeatherMap API Key
api_key = "26fea83d1259ec5c651394906a953b54"
base_url = "http://api.openweathermap.org/data/2.5/weather"
query_url = base_url + "?apikey=" + api_key + "&q=" + query_city
response = req.get(query_url)
json_data = response.json()

In [13]:
#Random selection of 500 unique cities based on lat/long coordinates
#Truncate random coordinates to 6 decimal points each
def truncate(x, d): #Source for truncate() function: https://stackoverflow.com/a/20545024
    return int(x*(10.0**d))/(10.0**d)
    #x = input
    #d = decimal places to truncate to

cityList = []
selected_cites = []
uniqueCityCount = 0

while (uniqueCityCount < 50): 
    random_lat = truncate(random.uniform(-90,90),6)
    random_long = truncate(random.uniform(-180,180),6)
    cityToCheck = citipy.nearest_city(random_lat, random_long).city_name
    #Pass query to OpenWeatherMap API to check if city is in database
    query_city = cityToCheck
    #Prevent duplicates from being appended to the cityList (before dataframe creation)
    #While also not adding the specified city if not in OWM database
    if cityToCheck not in cityList and json_data['cod'] != '404':
        cityList.append(cityToCheck)
        uniqueCityCount = uniqueCityCount+ 1 
        
#Create dataframe with uniqueCity names        
raw_cities_df = pd.DataFrame({"City Name": cityList})

#Not needed if duplicates removed before dataframe creation: 
#raw_cities_df = raw_cities_df.drop_duplicates() 

#Creation of column for city number
raw_cities_df['City Number'] = raw_cities_df.index + 1
raw_cities_df= raw_cities_df[['City Number','City Name']]
cn = pd.to_numeric(raw_cities_df['City Number'], errors='raise', downcast=None)

In [14]:

raw_cities_df.head(25)
#print(raw_cities_df['City Number'])

,City Number,City Name
0,1,lagos
1,2,avarua
2,3,vitim
3,4,changde
4,5,bredasdorp
5,6,albany
6,7,jamestown
7,8,puerto ayora
8,9,qaanaaq
9,10,barentsburg


In [15]:
settings = {"units": "imperial","appid": api_key}
from time import sleep

#Print parameters being requested from owm API before retrival
print("Base URL for API requests:... " + ow.BASE_URL)
print("Parameters being requested per city:... ")
summary = ["name", "coord.lat", "coord.lon","main.temp","main.humidity","wind.speed","clouds.all"]
print(summary)

#Get current weather for each city in cityList
cityCount = 1
for city in raw_cities_df["City Name"]:
    sleep(1.2)
    try:
        print("Processing city " + str(cityCount)+ " of " +str(len(raw_cities_df["City Name"])) + ", "+ city +":") 
        current_city= ow.get_current(city, **settings)
        sleep(1.2)
        #Leveraging OpenWatherMapy to parse the response with current_city(*summary)
        #summary = ["name", "coord.lat", "coord.lon","main.temp","main.humidity","wind.speed","clouds.all"]
        summary_output = current_city(*summary)
        print(str(summary_output))
    
    except Exception as ex:
        print(ex)
    cityCount = cityCount + 1

    #openweathermap complete city list:
    #http://openweathermap.org/help/city_list.txt 
    #one method ---> check against list
    #another method ---> skip if error returned 
    
    #https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_html.html
    
    

Base URL for API requests:... http://api.openweathermap.org/data/2.5/
Parameters being requested per city:... 
['name', 'coord.lat', 'coord.lon', 'main.temp', 'main.humidity', 'wind.speed', 'clouds.all']
Processing city 1 of 50, lagos:
('Lagos', 6.46, 3.39, 78.8, 65, 4.72, 8)
Processing city 2 of 50, avarua:
('Avarua', -21.21, -159.78, 77, 83, 11.41, 90)
Processing city 3 of 50, vitim:
('Vitim', 59.44, 112.57, 6, 71, 8.75, 88)
Processing city 4 of 50, changde:
('Changde', 29.04, 111.68, 31.43, 100, 13.44, 100)
Processing city 5 of 50, bredasdorp:
('Bredasdorp', -34.53, 20.04, 69.8, 83, 10.29, 0)
Processing city 6 of 50, albany:
('Albany', 42.65, -73.75, 22.73, 41, 6.93, 20)
Processing city 7 of 50, jamestown:
('Jamestown', -33.21, 138.6, 56.45, 72, 4.94, 0)
Processing city 8 of 50, puerto ayora:
('Puerto Ayora', -0.74, -90.35, 82.4, 58, 16.11, 20)
Processing city 9 of 50, qaanaaq:
('Qaanaaq', 77.48, -69.36, -16.91, 97, 6.73, 12)
Processing city 10 of 50, barentsburg:
HTTP Error 404: No

In [16]:
#Debugging for requests:
api_key = "26fea83d1259ec5c651394906a953b54"
base_url = "http://api.openweathermap.org/data/2.5/weather"
query_url = base_url + "?apikey=" + api_key + "&q=" + query_city
response = req.get(query_url)
json_data = response.json()
query_city= "sentyabrskiy"
print(json_data)

{'coord': {'lon': -35.46, 'lat': -5.2}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'base': 'stations', 'main': {'temp': 299.833, 'pressure': 1020.53, 'humidity': 81, 'temp_min': 299.833, 'temp_max': 299.833, 'sea_level': 1024.68, 'grnd_level': 1020.53}, 'wind': {'speed': 3.86, 'deg': 66.5}, 'clouds': {'all': 32}, 'dt': 1515014818, 'sys': {'message': 0.0083, 'country': 'BR', 'sunrise': 1514967257, 'sunset': 1515011954}, 'id': 3386213, 'name': 'Touros', 'cod': 200}


In [17]:
query_city= "belushya guba"
print(json_data)

{'coord': {'lon': -35.46, 'lat': -5.2}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'base': 'stations', 'main': {'temp': 299.833, 'pressure': 1020.53, 'humidity': 81, 'temp_min': 299.833, 'temp_max': 299.833, 'sea_level': 1024.68, 'grnd_level': 1020.53}, 'wind': {'speed': 3.86, 'deg': 66.5}, 'clouds': {'all': 32}, 'dt': 1515014818, 'sys': {'message': 0.0083, 'country': 'BR', 'sunrise': 1514967257, 'sunset': 1515011954}, 'id': 3386213, 'name': 'Touros', 'cod': 200}


In [18]:
query_city ="illoqqortoormiut"
print(json_data)

{'coord': {'lon': -35.46, 'lat': -5.2}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'base': 'stations', 'main': {'temp': 299.833, 'pressure': 1020.53, 'humidity': 81, 'temp_min': 299.833, 'temp_max': 299.833, 'sea_level': 1024.68, 'grnd_level': 1020.53}, 'wind': {'speed': 3.86, 'deg': 66.5}, 'clouds': {'all': 32}, 'dt': 1515014818, 'sys': {'message': 0.0083, 'country': 'BR', 'sunrise': 1514967257, 'sunset': 1515011954}, 'id': 3386213, 'name': 'Touros', 'cod': 200}


In [19]:
query_city ="mancio lima"
print(json_data)

{'coord': {'lon': -35.46, 'lat': -5.2}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'base': 'stations', 'main': {'temp': 299.833, 'pressure': 1020.53, 'humidity': 81, 'temp_min': 299.833, 'temp_max': 299.833, 'sea_level': 1024.68, 'grnd_level': 1020.53}, 'wind': {'speed': 3.86, 'deg': 66.5}, 'clouds': {'all': 32}, 'dt': 1515014818, 'sys': {'message': 0.0083, 'country': 'BR', 'sunrise': 1514967257, 'sunset': 1515011954}, 'id': 3386213, 'name': 'Touros', 'cod': 200}


In [21]:
query_city ="querbarentsburg"
print(json_data)

{'coord': {'lon': -35.46, 'lat': -5.2}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'base': 'stations', 'main': {'temp': 299.833, 'pressure': 1020.53, 'humidity': 81, 'temp_min': 299.833, 'temp_max': 299.833, 'sea_level': 1024.68, 'grnd_level': 1020.53}, 'wind': {'speed': 3.86, 'deg': 66.5}, 'clouds': {'all': 32}, 'dt': 1515014818, 'sys': {'message': 0.0083, 'country': 'BR', 'sunrise': 1514967257, 'sunset': 1515011954}, 'id': 3386213, 'name': 'Touros', 'cod': 200}
